Make this notebook as wide as the browser window allows:

In [82]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Functions to read the MessagePack binary file:

In [71]:
import msgpack
import numpy as np

def ext_hook(code, data):
    """Read code-2 data as float32 numpy arrays."""
    if code == 2:
        return np.frombuffer(data, dtype=np.float32)
    return ExtType(code, data)


def load_summary(path):
    """Load summary data from a file."""
    with open(path, "rb") as fp:
        raw_bytes = fp.read()

    return msgpack.unpackb(
        raw_bytes, strict_map_key=False, use_list=False, ext_hook=ext_hook
    )

Helper functions for querying summary data:

In [338]:
import datetime

def get_dates(summary):
    """Extract a list of dates from a summary."""
    time_data = summary["time"]
    if "YEAR" in time_data and "MONTH" in time_data and "DAY" in time_data:
        # easy path first
        return [
            datetime.datetime(y, m, d)
            for (y, m, d) in zip(
                time_data["YEAR"]["values"],
                time_data["MONTH"]["values"],
                time_data["DAY"]["values"],
            )
        ]
    else:
        # do all the work ourselves
        start_date = datetime.datetime(*reversed(summary["start_date"]))
        return [
            start_date + datetime.timedelta(seconds=int(d * 86400))
            for d in time_data["TIME"]["values"]
        ]
    
def common_keys(summaries, union=False):
    """Recursively extract common keys from a list of summaries."""
    summaries = [s for s in summaries if isinstance(s, dict) and "unit" not in s]
    if len(summaries) == 0:
        return None

    keys = set(summaries[0].keys())
    for s in summaries[1:]:
        if union:
            keys &= s.keys()
        else:
            keys |= s.keys()
    keys -= {'start_date', 'time'}

    return {k: common_keys([s.get(k, None) for s in summaries]) for k in keys}

Getting ready to plot:

In [89]:
import matplotlib.pyplot as plt

def plot_summary_keyword(summary, keys):
    """Plots data from a summary given a path of keys to extract it."""
    dates = get_dates(summary)
    
    for key in keys:
        summary = summary[key]
    values = summary['values']
    
    f, ax = plt.subplots(1, 1, figsize = (15, 10))
    ax.plot(dates, values, 'o-', markevery=0.02);
    ax.grid()
    ax.set_xlabel(f'Date');
    ax.set_ylabel(f'{keys[-1]} [{summary["unit"]}]');

    # adjust all font sizes in one sweeping motion
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(18)

This class will store all our summary data and selection widgets

In [357]:
from ipyfilechooser import FileChooser
import os
import ipywidgets as wg

# Jupyter widgets use traitlets
import traitlets as tts

# But Yan really likes traits
from traits.api import (
    Bool, cached_property, HasTraits, Dict, Instance, List, on_trait_change, Property, Tuple, Unicode)

TYPE_TO_BUTTON_VALUE = {
    "perf": "Perf",
    "field": "F",
    "regions": "R",
    "aquifers": "A",
    "wells": "W",
    "completions": "C",
    "groups": "G",
    "cells": "B"
}

NEED_LOCATION = {
    "regions",
    "aquifers",
    "wells",
    "completions",
    "groups",
    "cells"
}

class DataView(tts.HasTraits):
    # what we are viewing
    data_manager = tts.Instance(DataManager, ())
                
    fc = tts.Instance(
        FileChooser,
        kw = dict(
            use_dir_icons=True,
            show_hidden=False,
            title='<b>Select an .mpk file to open</b>')
    )
    
    file_selector = tts.Instance(
        wg.SelectMultiple,
        kw = dict(
            options=[],
            value=[],
            rows=5,
            description='Loaded Files:',
            disabled=False)
    )
    
    type_selector = tts.Instance(
        wg.ToggleButtons,
        kw = dict(
            options=[],
            description='Type:',
            disabled=True,
            style={'button_width': '40px'})
    )
    
    loc_selector = tts.Instance(
        wg.Dropdown,
        kw = dict(
            options=[],
            description='Location:',
            disabled=True)
    )
    
    kw_selector = tts.Instance(
        wg.Dropdown,
        kw = dict(
            options=[],
            description='Keyword:',
            disabled=True)
    )
    
    use_all_kws = tts.Instance(
        wg.Checkbox,
        kw = dict(
        value=False,
        description='List keywords from all files')
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # _select is the Select button of the FileChooser class
        self.fc._select.on_click(self._file_opened)
        self.file_selector.observe(self._file_selected, names='value')
        self.type_selector.observe(self._type_selected, names='value')
    
    # Event handlers
    def _file_opened(self, change):
        self.data_manager.add_summary(self.fc.selected)
        self.file_selector.options = [
            (os.path.basename(p), p) for p in self.data_manager.file_paths()
        ]
    
    def _file_selected(self, change):
        self.type_selector.disabled = False
        self.data_manager.selected_paths = self.file_selector.value
        self.type_selector.options = [
            (TYPE_TO_BUTTON_VALUE[k], k) for k in self.data_manager.common_keys
        ]
        
    def _type_selected(self, change):
        if change['new'] in NEED_LOCATION:
            pass
        else:
            pass


class DataManager(HasTraits):
    # Private fields
    _summary_data = Dict()
    _dates = Dict()
    
    all_keywords = Bool(False)
    selected_paths = Tuple(Unicode)
    
    common_keys = Property(depends_on = ['_summary_data, add_keywords, selected_paths'])
    
    def add_summary(self, path):
        if path is None: return
        self._summary_data[path] = load_summary(path)
        self._dates[path] =  get_dates(self._summary_data[path])
        
    def get_dates(self, path):
        return self._dates[path]
    
    def file_paths(self):
        return list(self._summary_data.keys())
    
    def unload_files(self, paths):
        for p in paths:
            del self._summary_data[p]
            del self._dates[p]
    
    @cached_property
    def _get_common_keys(self):
        return common_keys([self._summary_data[p] for p in self.selected_paths], self.all_keywords)
            
view = DataView()

In [361]:
data_selectors = ipywidgets.VBox([
    view.fc,
    view.file_selector,
    view.use_all_kws,
    view.type_selector, 
    view.loc_selector, 
    view.kw_selector],
    layout=wg.Layout(width='500px'))

display(data_selectors)